In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [11]:
pip install gym

Note: you may need to restart the kernel to use updated packages.


In [12]:
!apt-get update && apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)


In [1]:
import gym
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from stable_baselines3 import PPO

In [2]:
# import gymnasium as gym
import numpy as np

print(f"{gym.__version__=}")

gym.__version__='0.21.0'


In [3]:
!pip install gym_anytrading

In [4]:
# load preprocessed data from 10 nodes in the bay area

data = pd.read_csv(
    "processed.csv"
)

In [5]:
data = data.drop(
    [
        "Unnamed: 0",
        "INTERVALENDTIME_GMT",
        "OPR_DT",
        "OPR_HR",
        "NODE_ID_XML",
        "NODE_ID",
        "NODE",
        "MARKET_RUN_ID",
        "LMP_TYPE",
        "XML_DATA_ITEM",
        "PNODE_RESMRID",
        "GRP_TYPE",
    ],
    axis=1,
)

In [6]:
data["INTERVALSTARTTIME_GMT"] = pd.to_datetime(data["INTERVALSTARTTIME_GMT"])

In [7]:
data.set_index("INTERVALSTARTTIME_GMT", inplace=True)
data.head()

,POS,VALUE,OPR_INTERVAL,GROUP,HOUR,2DA,7DA,ACTUAL,DAM,ETo (in),Precip (in),Rel Hum (%),Wind Speed (mph),Soil Temp (F)
INTERVALSTARTTIME_GMT,,,,,,,,,,,,,,
2020-01-01 08:00:00+00:00,1,-0.422382,1,5,16,-4.146099,-4.113026,-4.485045,-4.307317,-3.023405,-0.03305,3.663296,-1.213782,0.188814
2020-01-01 08:05:00+00:00,1,-0.405158,2,5,16,-4.146099,-4.113026,-4.485045,-4.307317,-3.023405,-0.03305,3.663296,-1.213782,0.188814
2020-01-01 08:10:00+00:00,1,-0.417772,3,5,16,-4.146099,-4.113026,-4.485045,-4.307317,-3.023405,-0.03305,3.663296,-1.213782,0.188814
2020-01-01 08:15:00+00:00,1,-0.418884,4,5,16,-4.146099,-4.113026,-4.485045,-4.307317,-3.023405,-0.03305,3.663296,-1.213782,0.188814
2020-01-01 08:20:00+00:00,1,-0.414159,5,5,16,-4.146099,-4.113026,-4.485045,-4.307317,-3.023405,-0.03305,3.663296,-1.213782,0.188814


In [22]:
df = data

In [23]:
train_set_length = int(round((df.shape[0] * 0.75), 0))
test_set_length = df.shape[0] - train_set_length

print(f"\nTrain: {train_set_length} rows\nTest:  {test_set_length} rows")

train = df.iloc[:train_set_length]
test = df.iloc[train_set_length:]

print(f"\nTrain set shape: {train.shape}\nTest set shape:  {test.shape}\n")


Train: 2344464 rows
Test:  781488 rows

Train set shape: (2344464, 14)
Test set shape:  (781488, 14)



In [24]:
# Define the gym environment
class TimeSeriesEnv(gym.Env):
    def __init__(self, data):
        self.data = data
        self.action_space = gym.spaces.Discrete(700)
        self.observation_space = gym.spaces.Box(
            low=-np.inf, high=np.inf, shape=(len(data.columns),)
        )
        self.current_step = None

    def reset(self):
        self.current_step = 0
        obs = self.data.iloc[self.current_step].values
        return obs

    def step(self, action):
        assert self.action_space.contains(action)
        
        row = self.data.iloc[self.current_step]
        prediction = (action - 200) * 0.1
        reward = -abs(row['VALUE'] - prediction)
        
        self.current_step += 1
        
        done = self.current_step == len(self.data)
        if not done:
            obs = self.data.iloc[self.current_step].values
        else:
            obs = None
        return obs, reward, done, {}

In [25]:
# Create the gym environment
env = TimeSeriesEnv(train)

In [32]:
# Train the agent using PPO algorithm
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/zyan/miniconda3/envs/pytorch/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 1356 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1075         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0012191907 |
|    clip_fraction        | 0.000439     |
|    clip_range           | 0.2          |
|    entropy_loss         | -6.55        |
|    explained_variance   | 0.000638     |
|    learning_rate        | 0.0003       |
|    loss                 | 5.5e+04      |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0132      |
|    value_loss           | 1.14e+05     |
------------------------------------------
----------------

KeyboardInterrupt: 

In [33]:
from tqdm.notebook import tqdm

y_test = []
prediction = []

# Create the gym environment
env = TimeSeriesEnv(test)

# Test the agent
obs = env.reset()
for i in tqdm(range(157875)):
    action, _ = model.predict(obs)
    y_test.append(obs[1])
    prediction.append((action - 200) * 0.1)
    obs, reward, done, _ = env.step(action)
    if done:
        break

  0%|          | 0/157875 [00:00<?, ?it/s]

This code creates a custom TimeSeriesEnv environment class that uses the multivariate time series data as the observation space, and a binary action space. The step() function calculates the reward and determines whether the episode is done. Then, we train a PPO algorithm using the model.learn() function and test the agent on the environment. Finally, we plot the results to visualize the actions taken by the agent.

In [34]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

score = np.sqrt(mse(y_test, prediction))
print(f"RMSE Score on LinearRegression Test set: {score:0.3f}")
mae = mean_absolute_error(y_test, prediction)
print(f"MAE Score on LinearRegression Test set: {mae:0.3f}")
mape = mean_absolute_percentage_error(y_test, prediction)
print(f"MAPE Score on LinearRegression Test set: {mape:0.3f}")

RMSE Score on LinearRegression Test set: 1.330
MAE Score on LinearRegression Test set: 0.633
MAPE Score on LinearRegression Test set: 4.707
